# Text Analytics
### Web Scraper

---

### 1. Downloading and interpreting robots.txt

In [29]:
from urllib.request import urlopen
res = urlopen("https://www.kmutt.ac.th/robots.txt")
print(res.read().decode('utf-8'))


User-agent: *
Disallow: /wp-admin/
Allow: /wp-admin/admin-ajax.php

Sitemap: https://www.kmutt.ac.th/wp-sitemap.xml



In [30]:
import urllib.robotparser

rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.kmutt.ac.th/robots.txt")
rp.read()
rp.can_fetch("*","https://www.kmutt.ac.th/wp-sitemap.xml")

True

In [31]:
rp.site_maps()

['https://www.kmutt.ac.th/wp-sitemap.xml']

### Selenium

In [32]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import re
import pandas as pd
import os
from bs4 import BeautifulSoup

option = Options()

# disable image
prefs = {"profile.managed_default_content_settings.images": 2}
option.add_experimental_option("prefs", prefs)


path = os.getcwd()
service = Service(path + "//chromedriver.exe")

driver = webdriver.Chrome(service=service)
soup = BeautifulSoup(driver.page_source, 'html.parser')
#driver.implicitly_wait(5)

### Beautiful soup

In [40]:
# get url in url = "https://www.kmutt.ac.th/"
from selenium.webdriver.common.by import By
def find_links(url):
    driver.get(url)
    sleep(0.05)
    links = []
    link = driver.find_elements(By.TAG_NAME, "a")
    for i in link:
        if i.get_attribute("href") != None:
            #if not start with http let add url
            if not i.get_attribute("href").startswith("http"):
                links.append(url + i.get_attribute("href"))
                
            else:
                links.append(i.get_attribute("href"))
        else:
            pass
    return links

def find_link_in_list(links):
    all_links = [] 
    for i in links:
        try:
            l = find_links(i)
            all_links.extend(l)
            count += 1
            if len(links) == 0:
                break
        except:
             pass
    return all_links

keep = []
keep = find_links("https://www.kmutt.ac.th/")

result = []
result = find_link_in_list(keep)
# remove duplicate
result = list(dict.fromkeys(result))


In [42]:
len(result)

3009

### 1. Web Spider dataFrame, must at least comtain 2 columns ['form', 'to']
- from : The original URl cntains taht link tha first time
- to : The extracted URL

In [44]:
#split by '//'
link_split = []
for i in result:
    link_split.append(i.split('//'))

#split by '/'
link_split2 = []
for i in link_split:
    link_split2.append(i[1].split('/'))
    # if end with '/' then remove it
    if link_split2[-1][-1] == '':
        link_split2[-1].pop()

#link_split2

In [45]:
# accumulate all link until last
link_from = []
link_to = []
for i in link_split2:
    for j in range(len(i)-1):
        link_from.append('/'.join(i[:j+1]))
        link_to.append('/'.join(i[:j+2]))

# create dataframe
df_1 = pd.DataFrame({'from':link_from, 'to':link_to})
print(len(df_1))
df_1 = df_1.drop_duplicates()
print(len(df_1))
#save to csv
df_1.to_csv('link.csv', index=False)

7250
3226


In [46]:
df_1

,from,to
0,www.kmutt.ac.th,www.kmutt.ac.th/en
1,www.kmutt.ac.th/en,www.kmutt.ac.th/en/education
2,www.kmutt.ac.th/en/education,www.kmutt.ac.th/en/education/kmutt-life
3,www.kmutt.ac.th/en/education/kmutt-life,www.kmutt.ac.th/en/education/kmutt-life/tel:+6...
4,gmi.kmutt.ac.th,gmi.kmutt.ac.th/course
...,...,...
7240,eds.kmutt.ac.th/curriculum/courses-offered,eds.kmutt.ac.th/curriculum/courses-offered/seem
7241,www.facebook.com,www.facebook.com/allactivity?privacy_source=ac...
7244,www.kmutt.ac.th/internationalization/inter-news,www.kmutt.ac.th/internationalization/inter-new...
7247,www.kmutt.ac.th/internationalization/global-pa...,www.kmutt.ac.th/internationalization/global-pa...


### 2. Content DataFrame. Must at least contain 3 columns
- no : running number (index) of this table
- url : The URL (only those from the KMUTT website)
- content : Text content

In [47]:
def content(url):
    driver.get(url)
    sleep(0.05)
    content = driver.find_elements(By.TAG_NAME, "div")
    text = []
    for i in content:
        if i.text != '':
            text.append(i.text)
        else:
            text.append(None)
    return text

# get content
content_list = []
for i in result:
    try:
        content_list.append(content(i))
    except:
        content_list.append(None)
        pass

# create dataframe
df_2 = pd.DataFrame({'url':result, 'content':content_list})
df_2

,url,content
0,https://www.kmutt.ac.th/en/education/kmutt-lif...,[มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี\nKing M...
1,https://gmi.kmutt.ac.th/course,"[None, None, None, None, None, None, None, Non..."
2,https://www.kmutt.ac.th/internationalization/m...,[มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี\nKing M...
3,https://www.kmutt.ac.th/event/tiny-hotel/,[มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี\nKing M...
4,https://youtu.be/1TyM7hXRzhQ,None
...,...,...
3004,https://www.facebook.com/allactivity?privacy_s...,None
3005,https://www.kmutt.ac.th/internationalization/i...,None
3006,https://www.kmutt.ac.th/internationalization/g...,None
3007,https://www.kmutt.ac.th/news/7th-annual-confer...,None


In [ ]:
# save to csv
df_2.to_csv('content.csv', index=False)